A basic system:

User Query

   ↓

Decision (LLM)

   ↓

[Tool Call OR Direct Answer]

   ↓
   
Final Answer

In [41]:
#boring stuff
import os
from dotenv import load_dotenv

load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY missing in environment")

In [45]:
import faiss
import numpy as np
import os
import time
import random

from sentence_transformers import SentenceTransformer
from openai import OpenAI
from mistralai import Mistral

# ----------------------------
# 1. Documents
# ----------------------------

documents = [
    "RAG stands for Retrieval-Augmented Generation.",
    "In RAG, documents are embedded into a vector space.",
    "FAISS is a library for efficient similarity search.",
    "Large Language Models can hallucinate without grounding.",
    "Embeddings capture semantic meaning of text.",
    "Ronaldo has played for Real Madrid, Manchester United, Juventus, Sporting Lisbon, Portugal and Al Nassr."
]

# ----------------------------
# 2. Chunking
# ----------------------------

def chunk(text, size=50):
    words = text.split()
    return [" ".join(words[i:i+size]) for i in range(0, len(words), size)]

chunks = []
for d in documents:
    chunks.extend(chunk(d))

# ----------------------------
# 3. Embeddings
# ----------------------------
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(chunks).astype("float32")

# ----------------------------
# 4. FAISS index
# ----------------------------
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# ----------------------------
# 5. Retrieval tool
# ----------------------------
def retrieve(query, k=2):
    q_emb = embedder.encode([query]).astype("float32")
    _, idx = index.search(q_emb, k)
    return [chunks[i] for i in idx[0]]

# ----------------------------
# 6. LLM client 
# ----------------------------
openrouter = OpenAI( #Primary 
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

mistral = Mistral( # secondary
    api_key=os.getenv("MISTRAL_API_KEY", "")
)

# ----------------------------
# 7. Chat with fallback
# ----------------------------

def chat_with_fallback(messages, model_primary="stepfun/step-3.5-flash:free"):
    try:
        # PRIMARY: OpenRouter
        response = openrouter.chat.completions.create(
            model=model_primary,
            messages=messages,
        )
        return response.choices[0].message.content

    except Exception as e:
        print("⚠️ OpenRouter failed, falling back to Mistral:", e)

        # Optional small delay so you don't immediately re-hit limits
        time.sleep(1 + random.random())

        # FALLBACK: Mistral
        res = mistral.chat.complete(
            model="mistral-small-latest",
            messages=messages,
            stream=False,
        )
        return res.choices[0].message.content


# ----------------------------
# 8. Decision (biased toward retrieval for learning)
# ----------------------------

def decide_action(query):
    prompt = f"""
You are a routing agent.

Your job is to decide the action.

Valid actions:
- retrieve
- answer

Respond with ONLY ONE WORD.
No explanations.

Question:
{query}
"""

    raw = chat_with_fallback([
        {"role": "user", "content": prompt}
    ]).lower().strip()

    # HARD PARSING
    if "retrieve" in raw:
        return "retrieve"
    else:
        return "answer"


# ---------------------------- 
# 9. Answer step
# ----------------------------
def answer(query, context=None):
    if context:
        prompt = f"""
Answer using ONLY the context below. Answer in less than 50 words.
If the answer is not present, say "I don't know".

Context:
{context}

Question:
{query}
"""
    else:
        prompt = query

    return chat_with_fallback([
        {"role": "user", "content": prompt}
    ])

# ----------------------------
# 10. Agent loop
# ----------------------------

def agent(query):
    decision = decide_action(query)
    print("Decision:", decision)

    if decision == "retrieve":
        context_chunks = retrieve(query)
        context = "\n".join(context_chunks)
        print("Retrieved context:", context)
    else:
        context = None

    return answer(query, context)

# ----------------------------
# 11. Test
# ----------------------------
tests = [
    "Why are embeddings useful?",
    "What is RAG?",
    "What clubs did Ronaldo play for?",
    "How do I bake a cake?"
]

for t in tests:
    print("\nQUESTION:", t)
    print(agent(t))


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 266.20it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



QUESTION: Why are embeddings useful?
⚠️ OpenRouter failed, falling back to Mistral: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1770422400000'}, 'provider_name': None}}, 'user_id': 'user_33JOveQTDfa8tzJHX8UiRsza1LT'}
Decision: retrieve
Retrieved context: Embeddings capture semantic meaning of text.
In RAG, documents are embedded into a vector space.
⚠️ OpenRouter failed, falling back to Mistral: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1770422400000'}, 'provider_name': None}}, 'user_id': 'user_33JOveQTDfa8tzJHX8UiRsza1LT'}
Embeddings capture semantic meaning

Now we used LLM to make the decision. But this is wasteful, too much time and $. So we move to a score based system

User Query

   ↓

Vector similarity search

   ↓

If score > threshold → RAG

Else → direct answer


If best_distance < THRESHOLD → retrieve
Else → answer

In [46]:
#retrieval modified

def retrieve_with_scores(query, k=2):
    q_emb = embedder.encode([query]).astype("float32")
    distances, indices = index.search(q_emb, k)

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        results.append({
            "text": chunks[idx],
            "distance": float(dist)
        })

    return results


# distances

queries = [
    "What is RAG?",
    "What clubs did Ronaldo play for?",
    "How do I bake a cake?",
]

for q in queries:
    print("\nQuery:", q)
    for r in retrieve_with_scores(q):
        print(r)


Query: What is RAG?
{'text': 'RAG stands for Retrieval-Augmented Generation.', 'distance': 0.45085984468460083}
{'text': 'In RAG, documents are embedded into a vector space.', 'distance': 0.8678790926933289}

Query: What clubs did Ronaldo play for?
{'text': 'Ronaldo has played for Real Madrid, Manchester United, Juventus, Sporting Lisbon, Portugal and Al Nassr.', 'distance': 0.503292977809906}
{'text': 'FAISS is a library for efficient similarity search.', 'distance': 1.8585604429244995}

Query: How do I bake a cake?
{'text': 'Embeddings capture semantic meaning of text.', 'distance': 1.9504539966583252}
{'text': 'FAISS is a library for efficient similarity search.', 'distance': 1.9705548286437988}


Lower distances = better results

Lower = Retrieve, higher = answer

In [47]:
# pick a threshold
THRESHOLD = 1.2

# cheaper decision maker 

def should_retrieve(query, threshold=1.2):
    results = retrieve_with_scores(query, k=1)
    best_distance = results[0]["distance"]

    print("Best distance:", best_distance)

    return best_distance < threshold


# updated Agent loop

def agent(query):
    if should_retrieve(query):
        results = retrieve_with_scores(query)
        context = "\n".join(r["text"] for r in results)
        print("Decision: retrieve")
        print("Context:", context)
        return answer(query, context)
    else:
        print("Decision: answer")
        return answer(query)

In [27]:
# testing

tests = [
    "Why are embeddings useful?",
    "What is RAG?",
    "What clubs did Ronaldo play for?",
    "How do I bake a cake?",
]

for t in tests:
    print("\nQUESTION:", t)
    print(agent(t))



QUESTION: Why are embeddings useful?
Best distance: 0.9399771690368652
Decision: retrieve
Context: Embeddings capture semantic meaning of text.
In RAG, documents are embedded into a vector space.
Embeddings capture semantic meaning of text, allowing documents to be represented in a vector space for retrieval.

QUESTION: What is RAG?
Best distance: 0.45085984468460083
Decision: retrieve
Context: RAG stands for Retrieval-Augmented Generation.
In RAG, documents are embedded into a vector space.
RAG stands for Retrieval‑Augmented Generation, a technique where documents are embedded into a vector space to enable retrieval‑augmented generation.

QUESTION: What clubs did Ronaldo play for?
Best distance: 0.503292977809906
Decision: retrieve
Context: Ronaldo has played for Real Madrid, Manchester United, Juventus, Sporting Lisbon, Portugal and Al Nassr.
FAISS is a library for efficient similarity search.

Ronaldo played for Real Madrid, Manchester United, Juventus, Sporting Lisbon, Portugal, a